<pre>
10군데 핫 플레이스

지도위에 표시(튤팁: 장소이름, 팝업: 메세지)

아이콘 모양을 전부 다르게
</pre>

In [195]:
import folium
import pandas as pd
import requests
from urllib.parse import quote
import json

- Hot place 10
<pre>
코엑스
예술의전당
롯데월드
대모산
N서울타워
</pre>

In [196]:
hot_places = [
'코엑스',
'예술의전당',
'롯데월드',
'대모산',
'N서울타워',
'한강공원',
'경복궁',
'양재천',
'여의도',
'NC백화점'
]

In [197]:
filepath = '../04.지도시각화/roadapikey.txt'
with open(filepath) as file:
    road_key = file.read()

In [198]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApiJsonp.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'

In [199]:
add_list = []
for place in hot_places:
    url = f'{road_url}?{params1}&keyword={quote(place)}&resultType=json'
    result = requests.get(url)
    res = json.loads(result.text[1:-1])
    addr = res['results']['juso'][0]['roadAddr']
    add_list.append(addr)

In [200]:
add_list

['서울특별시 강남구 영동대로 513(삼성동)',
 '서울특별시 서초구 남부순환로 2406(서초동)',
 '서울특별시 송파구 올림픽로 300(신천동)',
 '서울특별시 강남구 개포로 지하522(일원동)',
 '서울특별시 용산구 남산공원길 105(용산동2가)',
 '서울특별시 강동구 선사로 83-106(암사동)',
 '서울특별시 서대문구 북아현로22마길 21-8(북아현동, 경복궁아침)',
 '서울특별시 강남구 양재천로 163(도곡동)',
 '서울특별시 영등포구 63로 10(여의도동)',
 '서울특별시 구로구 구로중앙로 152(구로동)']

In [201]:
df = pd.DataFrame({'장소':hot_places, '도로명주소':add_list})
df

,장소,도로명주소
0,코엑스,서울특별시 강남구 영동대로 513(삼성동)
1,예술의전당,서울특별시 서초구 남부순환로 2406(서초동)
2,롯데월드,서울특별시 송파구 올림픽로 300(신천동)
3,대모산,서울특별시 강남구 개포로 지하522(일원동)
4,N서울타워,서울특별시 용산구 남산공원길 105(용산동2가)
5,한강공원,서울특별시 강동구 선사로 83-106(암사동)
6,경복궁,"서울특별시 서대문구 북아현로22마길 21-8(북아현동, 경복궁아침)"
7,양재천,서울특별시 강남구 양재천로 163(도곡동)
8,여의도,서울특별시 영등포구 63로 10(여의도동)
9,NC백화점,서울특별시 구로구 구로중앙로 152(구로동)


In [202]:
kakao_filepath = 'D:/이젠아카데미/02.DataAnalysis/04.지도시각화/kakaoapikey.txt'
with open(kakao_filepath) as file:
    kakao_key = file.read()

In [203]:
local_url = "https://dapi.kakao.com/v2/local/search/address.json"
lng_list, lat_list = [],[]
for i in df.index:
    url = f'{local_url}?query={quote(df.도로명주소[i])}'
    header = {'Authorization': f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    lng_list.append(float(result['documents'][0]['x']))
    lat_list.append(float(result['documents'][0]['y']))
df['위도'] = lat_list
df['경도'] = lng_list
df

,장소,도로명주소,위도,경도
0,코엑스,서울특별시 강남구 영동대로 513(삼성동),37.511824,127.059159
1,예술의전당,서울특별시 서초구 남부순환로 2406(서초동),37.480240,127.014215
2,롯데월드,서울특별시 송파구 올림픽로 300(신천동),37.513752,127.104447
3,대모산,서울특별시 강남구 개포로 지하522(일원동),37.491545,127.073169
4,N서울타워,서울특별시 용산구 남산공원길 105(용산동2가),37.551123,126.987868
5,한강공원,서울특별시 강동구 선사로 83-106(암사동),37.549966,127.122249
6,경복궁,"서울특별시 서대문구 북아현로22마길 21-8(북아현동, 경복궁아침)",37.565647,126.955286
7,양재천,서울특별시 강남구 양재천로 163(도곡동),37.481320,127.044034
8,여의도,서울특별시 영등포구 63로 10(여의도동),37.518271,126.936730
9,NC백화점,서울특별시 구로구 구로중앙로 152(구로동),37.501160,126.882773


In [204]:
# 경복궁.png, 백화점.png | glob 같은 이름들의 파일들을 모음
import glob
for fname in glob.glob('*.png'):
    print(fname)
img = folium.features.CustomIcon('경복궁.png', icon_size=(50,50))
img2 = folium.features.CustomIcon('백화점.png', icon_size=(50,50))

경복궁.png
백화점.png


In [205]:
def color_img(i):
    if df.장소[i] == '경복궁':
        return img
    elif df.장소[i] == 'NC백화점':
        return img2
    else:
        return folium.Icon(color=colors[i], icon=icons[i])

In [206]:
colors = ['red','cadetblue','blue','green','black',
          'orange','경복궁','gray','purple','NC백화점']
icons = ['glyphicon-music','glyphicon-picture','glyphicon-star-empty','glyphicon-tree-conifer','glyphicon-glass',
         'glyphicon-send','경복궁','glyphicon-leaf','glyphicon-cutlery','NC백화점']

map = folium.Map(
    location=[df.위도.mean(), df.경도.mean()], zoom_start=12
)

for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        popup = folium.Popup(df.도로명주소[i], max_width=200),
        tooltip=df.장소[i],
        icon = color_img(i)
    ).add_to(map)

title_html = '<h3 align="center" style="font-size:20px">근처 5개 Hot Places</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map

- 글자를 써 봅시다.

In [212]:
from folium.features import DivIcon

map = folium.Map(
    location=[df.위도.mean(), df.경도.mean()], zoom_start=12
)

folium.Marker(
        location=[df.위도[6], df.경도[6]],
        popup = folium.Popup(df.도로명주소[6], max_width=200),
        tooltip=df.장소[6],
        icon = folium.features.CustomIcon('경복궁.png', icon_size=(80,80))
).add_to(map)

folium.Marker(
        location=[df.위도[6], df.경도[6]],
        icon = DivIcon(icon_size=(80,20), icon_anchor=(20,0),
                html=f'<div style="font-size: 50pt"><bold>나만의 글씨</bold></div>')
).add_to(map)
map